<a href="https://colab.research.google.com/github/YinDongFang/ipynb/blob/main/funasr-demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
!pip install --upgrade --quiet pip
!pip install --upgrade --quiet datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio funasr tqdm --use-deprecated=legacy-resolver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 96.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 118.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 35.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.5/701.5 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 125.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 1

In [6]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice = load_dataset("urarik/thchs30", split="test[:100]")

print(common_voice)

train-00000-of-00008.parquet:  43%|####3     | 189M/434M [00:00<?, ?B/s]

train-00001-of-00008.parquet:   0%|          | 0.00/436M [00:00<?, ?B/s]

train-00002-of-00008.parquet:   0%|          | 0.00/439M [00:00<?, ?B/s]

train-00003-of-00008.parquet:   0%|          | 0.00/439M [00:00<?, ?B/s]

train-00004-of-00008.parquet:   0%|          | 0.00/439M [00:00<?, ?B/s]

...

noise-00004-of-00005.parquet:   0%|          | 0.00/440M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12049 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1339 [00:00<?, ? examples/s]

Generating noise split:   0%|          | 0/7485 [00:00<?, ? examples/s]

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 100
})


In [10]:
from tqdm import tqdm  # 用于显示进度条
import numpy as np

dataset = common_voice
predictions = []  # 存储模型的预测文本
references = []     # 存储参考文本 (ground truth)

print("正在进行语音识别... (仅限前 100 条数据)") # 修改提示信息，说明只处理前 100 条
for i in tqdm(range(10)): # 使用索引 range(100) 来迭代
    item = dataset[i] # 使用索引 i 访问数据集中的数据项 (应该是一个字典)
    reference_text = item["sentence"]

    res = model.generate(
        input=item["audio"]["array"],
        cache={},
        language="auto",  # "zn", "en", "yue", "ja", "ko", "nospeech"
        use_itn=True,
        batch_size_s=60,
        merge_vad=True,  #
        merge_length_s=15,
    )
    predicted_text = rich_transcription_postprocess(res[0]["text"])

    predictions.append(predicted_text)
    references.append(reference_text)

print("语音识别完成！ (前 100 条数据)") # 修改提示信息

正在进行语音识别... (仅限前 100 条数据)


  0%|          | 0/10 [00:00<?, ?it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00,  5.01it/s]


{'load_data': '0.000', 'extract_feat': '0.070', 'forward': '0.200', 'batch_size': '1', 'rtf': '0.024'}, : 100%|██████████| 1/1 [00:00<00:00,  5.01it/s]


rtf_avg: 0.024: 100%|██████████| 1/1 [00:00<00:00,  4.63it/s]



  0%|          | 0/1 [00:00<?, ?it/s]



  0%|          | 0/1 [00:00<?, ?it/s]



100%|██████████| 1/1 [00:03<00:00,  3.18s/it]



{'load_data': '0.000', 'extract_feat': '0.026', 'forward': '3.181', 'batch_size': '1', 'rtf': '0.399'}, : 100%|██████████| 1/1 [00:03<00:00,  3.18s/it]



rtf_avg: 0.399: 100%|██████████| 1/1 [00:03<00:00,  3.20s/it]



100%|██████████| 1/1 [00:03<00:00,  3.25s/it]


rtf_avg: 0.393, time_speech:  8.250, time_escape: 3.246: 100%|██████████| 1/1 [00:03<00:00,  3.25s/it]
 10%|█         | 1/10 [00:03<00:31,  3.49s/it]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00,  5.18it/s]


{'load_data':

语音识别完成！ (前 10 条数据)


In [1]:
import pandas as pd

data = pd.DataFrame(dict(predictions=predictions, reference=references))
data

NameError: name 'predictions' is not defined